In [8]:
%load_ext autoreload
%autoreload 2

In [50]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from im2mesh import config, data
import torch
import pprint

In [11]:
os.chdir('..')

In [63]:
cfg = config.load_config('configs/domain_adaptation/chair_DA_DANN.yaml', 'configs/default.yaml')
train_dataset = config.get_dataset('train', cfg)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=3, num_workers=1, shuffle=True,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)

In [64]:
for batch in train_loader:
    for key in batch:
        print("{}-->{}".format(key, batch[key].shape))
    break

Error occured when loading field inputs_target_domain of model a3e3eee0669bec82c9f016d57db96408


The error was:empty range for randrange() (0,0, 0)


Error occured when loading field inputs_target_domain of model 4447ee47cd2333f6349bb1cbbf9a4206


The error was:empty range for randrange() (0,0, 0)


Error occured when loading field inputs_target_domain of model 904ddb7762488b1b5dc3653f8341633a


...

The error was:empty range for randrange() (0,0, 0)


Error occured when loading field inputs_target_domain of model 12f4d8c20ef311d988dcbe86402c7c15


The error was:empty range for randrange() (0,0, 0)


Error occured when loading field inputs_target_domain of model 715530026c6459873e8366d7fa2c218f


The error was:empty range for randrange() (0,0, 0)
